In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

khadiza13_less_data_path = kagglehub.dataset_download('khadiza13/less-data')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/less-data/changed_ds/new_ds_small.csv")
df.head()

,image,text,label
0,2001.jpg,আচ্ছা ভাই।\n মেয়েদের থেকে দূরে থাকবা মেয়ের পাল...,stereotype & objectification
1,2002.jpg,কিসের foodpanda যেখানে আমার সাদিয়া রে অর্ডার ক...,stereotype & objectification
2,2003.jpg,উপস্থিত sir\n Yes sir\n Present Teacher\n [লাব...,stereotype & objectification
3,2004.png,আমি হিজাব চাইনি\n তারপর আমার স্বামী আমাকে বোঝালেন,violence or abuse
4,2005.jpg,"বাঙ্গু (beta, omega, theta, delta) male\nসুন্দ...",non-misogynistic


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import random
from tqdm import tqdm

# Set random seed for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
set_seed(42)

# Load dataset
df = pd.read_csv("/kaggle/input/less-data/changed_ds/new_ds_small.csv")
assert not df.empty, "Dataset is empty"
assert len(df) == 4001, f"Expected 4001 samples, got {len(df)}"
assert df['image'].notnull().all(), "Missing values in 'image' column"
assert df['label'].notnull().all(), "Missing values in 'label' column"
print("✅ Dataset loaded. Sample:")
print(df.head())

# Encode labels
label_encoder = LabelEncoder()
df['label_id'] = label_encoder.fit_transform(df['label'])
label2id = {label: idx for idx, label in enumerate(label_encoder.classes_)}
id2label = {idx: label for label, idx in label2id.items()}
print(f"✅ Unique labels: {label2id}")
print(f"Label distribution: {df['label'].value_counts().to_dict()}")

# Split data into train (70%), validation (15%), test (15%)
train_df, temp_df = train_test_split(
    df, test_size=0.3, random_state=42, stratify=df['label_id']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label_id']
)
print(f"✅ Dataset split: Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Image dataset class
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.image_paths = df['image'].values
        self.labels = df['label_id'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_paths[idx])
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (224, 224))  # Fallback: blank image
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return {'image': image, 'labels': label}

# Image preprocessing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
img_dir = "/kaggle/input/less-data/changed_ds/img"
train_dataset = ImageDataset(train_df, img_dir, transform=train_transform)
val_dataset = ImageDataset(val_df, img_dir, transform=val_test_transform)
test_dataset = ImageDataset(test_df, img_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define custom CNN
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Model definitions
def get_model(model_name, num_classes):
    if model_name == "CustomCNN":
        model = CustomCNN(num_classes)
    elif model_name == "ResNet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "VGG16":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name == "VGG19":
        model = models.vgg19(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    return model

# Training and evaluation function
def train_and_evaluate(model_name, model, train_loader, val_loader, test_loader, num_classes, device):
    print(f"\n🔄 Training {model_name}...")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    # Early stopping parameters
    patience = 3
    min_delta = 0.01
    best_f1 = 0
    patience_counter = 0

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        # Progress bar for training
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for batch in train_bar:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_bar.set_postfix({"loss": running_loss / len(train_bar)})

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        val_loss = 0.0
        # Progress bar for validation
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1} Validation")
        with torch.no_grad():
            for batch in val_bar:
                images = batch['image'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_bar.set_postfix({"val_loss": val_loss / len(val_bar)})

        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        print(f"Epoch {epoch+1}: Val F1 = {val_f1:.4f}")

        # Early stopping
        if val_f1 > best_f1 + min_delta:
            best_f1 = val_f1
            patience_counter = 0
            os.makedirs(f"./models/{model_name}", exist_ok=True)
            torch.save(model.state_dict(), f"./models/{model_name}/best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Load best model
    model.load_state_dict(torch.load(f"./models/{model_name}/best_model.pt"))

    # Validation evaluation
    model.eval()
    val_preds, val_labels = [], []
    # Progress bar for validation evaluation
    val_bar = tqdm(val_loader, desc="Validation Evaluation")
    with torch.no_grad():
        for batch in val_bar:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    print(f"\n📊 Validation Classification Report for {model_name}")
    val_report = classification_report(val_labels, val_preds, target_names=label_encoder.classes_)
    print(val_report)

    # Test evaluation
    test_preds, test_labels = [], []
    # Progress bar for test evaluation
    test_bar = tqdm(test_loader, desc="Test Evaluation")
    with torch.no_grad():
        for batch in test_bar:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    print(f"\n📊 Test Classification Report for {model_name}")
    test_report = classification_report(test_labels, test_preds, target_names=label_encoder.classes_)
    print(test_report)

    # Save results
    results[model_name] = {
        "val_report": classification_report(val_labels, val_preds, target_names=label_encoder.classes_, output_dict=True),
        "test_report": classification_report(test_labels, test_preds, target_names=label_encoder.classes_, output_dict=True)
    }

    # Clear memory
    del model
    torch.cuda.empty_cache()
    print(f"🧹 Cleared memory after {model_name}")

# Main execution
results = {}
model_names = ["CustomCNN", "ResNet50", "VGG16", "VGG19"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Train and evaluate models
for model_name in model_names:
    try:
        model = get_model(model_name, num_classes=len(label2id))
        train_and_evaluate(model_name, model, train_loader, val_loader, test_loader, len(label2id), device)
    except Exception as e:
        print(f"❌ Skipping {model_name} due to error: {e}")

# Compare results
print("\n📊 Model Comparison (Test Metrics)")
for model_name, result in results.items():
    test_acc = result["test_report"]["accuracy"]
    test_f1 = result["test_report"]["weighted avg"]["f1-score"]
    print(f"{model_name}: Accuracy = {test_acc:.4f}, F1 = {test_f1:.4f}")

✅ Dataset loaded. Sample:
      image                                               text  \
0  2001.jpg  আচ্ছা ভাই।\n মেয়েদের থেকে দূরে থাকবা মেয়ের পাল...   
1  2002.jpg  কিসের foodpanda যেখানে আমার সাদিয়া রে অর্ডার ক...   
2  2003.jpg  উপস্থিত sir\n Yes sir\n Present Teacher\n [লাব...   
3  2004.png  আমি হিজাব চাইনি\n তারপর আমার স্বামী আমাকে বোঝালেন   
4  2005.jpg  বাঙ্গু (beta, omega, theta, delta) male\nসুন্দ...   

                          label  
0  stereotype & objectification  
1  stereotype & objectification  
2  stereotype & objectification  
3             violence or abuse  
4              non-misogynistic  
✅ Unique labels: {'non-misogynistic': 0, 'stereotype & objectification': 1, 'violence or abuse': 2}
Label distribution: {'stereotype & objectification': 1591, 'non-misogynistic': 1380, 'violence or abuse': 1030}
✅ Dataset split: Train: 2800, Val: 600, Test: 601
Using device: cuda

🔄 Training CustomCNN...


Epoch 1 Training:  60%|██████    | 105/175 [00:27<00:17,  3.94it/s, loss=0.662]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1 Validation: 100%|██████████| 38/38 [00:08<00:00,  4.55it/s, val_loss=1.01] 


Epoch 1: Val F1 = 0.3881


Epoch 2 Training:  24%|██▍       | 42/175 [00:10<00:35,  3.76it/s, loss=0.249] /usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 2 Validation: 100%|██████████| 38/38 [00:08<00:00,  4.48it/s, val_loss=0.99] 


Epoch 2: Val F1 = 0.4122


...

Epoch 8: Val F1 = 0.5643
Early stopping at epoch 8


Validation Evaluation: 100%|██████████| 38/38 [00:09<00:00,  3.91it/s]



📊 Validation Classification Report for VGG19
                              precision    recall  f1-score   support

            non-misogynistic       0.64      0.69      0.66       207
stereotype & objectification       0.58      0.64      0.61       239
           violence or abuse       0.44      0.33      0.38       154

                    accuracy                           0.57       600
                   macro avg       0.55      0.55      0.55       600
                weighted avg       0.56      0.57      0.57       600



Test Evaluation: 100%|██████████| 38/38 [00:10<00:00,  3.49it/s]



📊 Test Classification Report for VGG19
                              precision    recall  f1-score   support

            non-misogynistic       0.60      0.71      0.65       207
stereotype & objectification       0.55      0.59      0.57       239
           violence or abuse       0.45      0.28      0.35       155

                    accuracy                           0.55       601
                   macro avg       0.53      0.53      0.52       601
                weighted avg       0.54      0.55      0.54       601

🧹 Cleared memory after VGG19

📊 Model Comparison (Test Metrics)
CustomCNN: Accuracy = 0.5191, F1 = 0.5092
ResNet50: Accuracy = 0.6223, F1 = 0.6140
VGG16: Accuracy = 0.5624, F1 = 0.5525
VGG19: Accuracy = 0.5541, F1 = 0.5409
